In [231]:
import codecs
from lxml import html
import requests
from urlparse import parse_qs, urlparse

In [228]:
def ensure_unicode(s):
    if isinstance(s, unicode):
        return s
    else:
        return s.decode('utf-8')

In [220]:
project_titles = []

In [223]:
page = requests.get("https://tools.wmflabs.org/enwp10/cgi-bin/pindex.fcgi?sec=[All]")
parser = html.HTMLParser(encoding='utf-8')
tree = html.document_fromstring(page.content, parser=parser)

In [224]:
cells = tree.xpath("//table[@class='wikitable']//tr/td[1]")

In [225]:
for cell in cells:
    if (len(cell[0]) == 0):
        title = ensure_unicode(cell[0].text)
        project_titles.append((title,title))
    else:
        title = ensure_unicode(cell[0][0].text)
        url = ensure_unicode(cell[0][0].attrib['href'])
        query = urlparse(url).query
        unique = parse_qs(query)['title'][0]
        project_titles.append((title,unique))

In [226]:
project_titles

In [96]:
def escape_csv(s):
    if ',' in s:
        s = s.replace('\\', '\\\\').replace('"', '\\"')
        return u'"%s"' % s
    else:
        return s

In [232]:
with open('data/projects.tsv', 'wb') as f:
    f.write(u"Title\tUnique\n".encode('utf_16_le'))
    for project in project_titles:
        title = project[0]
        unique = project[1]
        line = u"%s\t%s\n" % project
        f.write(line.encode('utf_16_le'))